In [1]:
using Pkg
Pkg.activate(".")
caminho_do_pacote = "/home/luizfreire/Documents/mestrado/powerModelsExemplos/PowerModelsDistributionDev"
Pkg.develop(PackageSpec(path=caminho_do_pacote))

  Activating project at `~/Documents/mestrado/powerModelsExemplos`
   Resolving package versions...
  No Changes to `~/Documents/mestrado/powerModelsExemplos/Project.toml`
  No Changes to `~/Documents/mestrado/powerModelsExemplos/Manifest.toml`


In [2]:
using PowerModelsDistribution
using Ipopt
using JuMP
using PowerPlots
using Plots

include("utils/load_data.jl")
results_path = "results/2025-04-21_solar_carga_armazenamento_opf_cost/"

solver = optimizer_with_attributes(
    Ipopt.Optimizer,
    "max_iter" => 50000,
    "tol" => 1e-8)


Precompiling PowerModelsDistribution
  ✓ PowerModelsDistribution
  1 dependency successfully precompiled in 11 seconds. 71 already precompiled.


MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("max_iter") => 50000, MathOptInterface.RawOptimizerAttribute("tol") => 1.0e-8])

In [3]:
eng_model = PowerModelsDistribution.parse_file("PowerModelsDistributionDev/test/data/opendss/case_tabulation.dss")
    
                        
add_prosumer!(eng_model,
              "prosumer_1",
              "bus_1",
              [1, 2, 3, 4];
              configuration=WYE,
              pg=[2500, 2500, 2500, 0],
              pd=[1800, 1800, 1800, 0],
              ps=[1000, 1000, 1000, 0],
              energy=20000,
              energy_ub=100000,
              charge_ub=7000,
              discharge_ub=7000,
              charge_efficiency=100,
              discharge_efficiency=100,
              cost=1)


[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 1 in 'case_tabulation.dss'
[ PowerModelsDistribution | Info ] : Command 'solve' on line 5 in 'case_tabulation.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : basemva=100 is the default value, you may want to adjust sbase_default for better convergence


Dict{String, Any} with 16 entries:
  "pg"                   => [2500, 2500, 2500, 0]
  "connections"          => [1, 2, 3, 4]
  "energy"               => 20000
  "bus"                  => "bus_1"
  "charge_ub"            => 7000
  "configuration"        => WYE
  "energy_ub"            => 100000
  "status"               => ENABLED
  "source_id"            => "prosumer.prosumer_1"
  "charge_efficiency"    => 100
  "discharge_ub"         => 7000
  "ps"                   => [1000, 1000, 1000, 0]
  "cost"                 => 1
  "control_mode"         => FREQUENCYDROOP
  "discharge_efficiency" => 100
  "pd"                   => [1800, 1800, 1800, 0]

In [5]:
result = solve_mc_model(eng_model, ACPUPowerModel, solver, build_mc_tes_opf)

[ PowerModelsDistribution | Info ] : no active generators found at bus 1, updating to bus type from 2 to 1



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      315
Number of nonzeros in inequality constraint Jacobian.:       42
Number of nonzeros in Lagrangian Hessian.............:      750

Total number of variables............................:       53
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        8
                     variables with only upper bounds:        0
Total number of equality constraints.................:       45
Total number of inequality c

Dict{String, Any} with 8 entries:
  "solve_time"         => 5.2369
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => -0.28
  "solution"           => Dict{String, Any}("voltage_source"=>Dict{String, Any}…
  "objective_lb"       => -Inf

In [6]:
teste = instantiate_mc_model(
    eng_model,
    ACPUPowerModel,
    build_mc_tes_opf)

[ PowerModelsDistribution | Info ] : Converting ENGINEERING data model to MATHEMATICAL first to build JuMP model
[ PowerModelsDistribution | Info ] : no active generators found at bus 1, updating to bus type from 2 to 1


ACPUPowerModel(A JuMP Model
├ solver: none
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 56
├ num_constraints: 91
│ ├ NonlinearExpr in MOI.EqualTo{Float64}: 24
│ ├ AffExpr in MOI.EqualTo{Float64}: 3
│ ├ AffExpr in MOI.GreaterThan{Float64}: 6
│ ├ AffExpr in MOI.LessThan{Float64}: 6
│ ├ QuadExpr in MOI.EqualTo{Float64}: 18
│ ├ QuadExpr in MOI.LessThan{Float64}: 6
│ ├ VariableRef in MOI.GreaterThan{Float64}: 17
│ └ VariableRef in MOI.LessThan{Float64}: 11
└ Names registered in the model: none, Dict{String, Any}("is_kron_reduced" => true, "conductor_ids" => [1, 2, 3], "time_elapsed" => 1.0, "bus" => Dict{String, Any}("1" => Dict{String, Any}("vm_pair_lb" => Tuple{Any, Any, Real}[], "grounded" => Bool[0, 0, 0], "vm_pair_ub" => Tuple{Any, Any, Real}[], "bus_i" => 1, "name" => "bus_1", "bus_type" => 1, "terminals" => [1, 2, 3], "vmax" => [Inf, Inf, Inf], "vbase" => 66.39528095680697, "source_id" => "bus.bus_1"…), "2" => Dict{String, Any}("vm_pair_lb" => Tu

In [7]:
nws(teste)[0][:prosumer]

Dict{Int64, Any} with 1 entry:
  1 => Dict{String, Any}("cdis"=>0, "energy_rating"=>1.0, "pg"=>[0.025, 0.025, …

In [8]:
teste_2 = transform_data_model(eng_model)["prosumer"]

[ PowerModelsDistribution | Info ] : no active generators found at bus 1, updating to bus type from 2 to 1


Dict{String, Any} with 1 entry:
  "1" => Dict{String, Any}("cdis"=>0, "energy_rating"=>1.0, "pg"=>[0.025, 0.025…

In [9]:
get(0, :prosumer, teste_2)

LoadError: MethodError: no method matching get(::Int64, ::Symbol, ::Dict{String, Any})

[0mClosest candidates are:
[0m  get([91m::REPL.Terminals.TTYTerminal[39m, ::Any, ::Any)
[0m[90m   @[39m [35mREPL[39m [90m/opt/julia-1.10.0/share/julia/stdlib/v1.10/REPL/src/[39m[90m[4mTerminals.jl:161[24m[39m
[0m  get([91m::Plots.Axis[39m, ::Symbol, ::Any)
[0m[90m   @[39m [36mPlots[39m [90m~/.julia/packages/Plots/FFuQi/src/[39m[90m[4margs.jl:1784[24m[39m
[0m  get([91m::Pkg.Types.Manifest[39m, ::Any, ::Any)
[0m[90m   @[39m [32mPkg[39m [90m/opt/julia-1.10.0/share/julia/stdlib/v1.10/Pkg/src/[39m[90m[4mTypes.jl:311[24m[39m
[0m  ...
